<a href="https://colab.research.google.com/github/paul-emile1/TIPE/blob/main/TIPE_sp%C3%A9_classement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**INTRODUCTION ET DEBUT DU TRAITEMENT DES DONNEES**
Création d'un classement de tennis grâce à un algortime de Perron-Frobenius.




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from datetime import timedelta
from datetime import *
from random import *
import copy
from math import *
from time import *
from datetime import timedelta, datetime
color = sns.color_palette()
sns.set_style('darkgrid')

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Chargement du dataset/supression des colonnes avec des données manquantes

data = pd.read_csv('/content/drive/MyDrive/TIPE/DATASET_INITIAL.csv')
data = data.dropna()
data = data.reset_index(drop=True)

In [ ]:
#Convertissement des données en type int

data['Player1Rank'] = pd.to_numeric(data['Player1Rank'])
data['Player2Rank'] = pd.to_numeric(data['Player2Rank'])
data['Player1Sets'] = pd.to_numeric(data['Player2Sets'])
data['Date'] = pd.to_datetime(data.Date)

In [ ]:
winner = data['Winner'].copy()

In [ ]:
#Player 1 = mieux classé ATP

def rename(data):
  for k in range(data.shape[0]):
    if data.iloc[k,5]>data.iloc[k,12]:
      data.iloc[k,0],data.iloc[k,1]=data.iloc[k,1],data.iloc[k,0]
      data.iloc[k,4],data.iloc[k,11]=data.iloc[k,11],data.iloc[k,4]
      data.iloc[k,5],data.iloc[k,12]=data.iloc[k,12],data.iloc[k,5]
      data.iloc[k,6],data.iloc[k,13]=data.iloc[k,13],data.iloc[k,6]
      data.iloc[k,7],data.iloc[k,14]=data.iloc[k,14],data.iloc[k,7]
  return data

In [ ]:
data = rename(data)

In [ ]:
data['Winner']= winner

In [ ]:
#Colonne Winner : 1 si P1 gagne 0 sino,
def win(data):
  for k in range(data.shape[0]):
    m = min(data.iloc[k, 5], data.iloc[k, 12])
    # Si P1 gagne...
    if data.iloc[k,6]==data.iloc[k, 15]:
      data.iloc[k,15] = 1
    else:
      data.iloc[k, 15] = 0
  return data

data = win(data)

### **Transformation de certaines variables en variables muettes (dummy variables)**

In [ ]:
#Transformation des variables round et surface en variables muettes
def round_number(x):
    if x == '1st Round':
        return 1
    elif x == '2nd Round':
        return 2
    elif x == '3rd Round':
        return 3
    elif x == '4th Round':
        return 4
    elif x == 'Quarterfinals':
        return 5
    elif x == 'Semifinals':
        return 6
    elif x == 'The Final':
        return 7

data['Round'] = data['Round'].apply(round_number)

dummy_ranks = pd.get_dummies(data['Round'], prefix='Round')

data = data.join(dummy_ranks.loc[:, 'Round_1':])
data[['Round_1','Round_2', 'Round_3',
       'Round_4', 'Round_5', 'Round_6', 'Round_7']] = data[['Round_1.0','Round_2.0', 'Round_3.0','Round_4.0', 'Round_5.0', 'Round_6.0', 'Round_7.0']].astype('int_')
data = data.drop(['Round_1.0','Round_2.0', 'Round_3.0','Round_4.0', 'Round_5.0', 'Round_6.0', 'Round_7.0'],axis = 1)

In [ ]:
dummy_ranks = pd.get_dummies(data['Surface'], prefix='Surface')

In [ ]:
data = data.join(dummy_ranks.loc[:, 'Surface_Clay':])
data = data.drop(['Surface'], axis = 1)
data[['Surface_Clay','Surface_Grass','Surface_Hard']] = data[['Surface_Clay','Surface_Grass','Surface_Hard']].astype('int_')
data = data.drop(["Round"],axis = 1)

###**Indexation des joueurs : création d'un dictionnaire contenant en clé le nom de joueur et en valeur : [index du joueur,son nombre de victoire, total de matchs joués, meilleur classement en carrière]**

In [ ]:
###peut-être une meilleure manière d'encoder les joueurs
index_joueurs_stats = {}
#Dictionnaire contenant le nom des joueurs comme une clé [player_index,x,y,z,t,w]
#x : nombres_de_matchs_gagnés
#y : nombres_de_matchs_joués
#z : meilleur classement en carrière (on le fixe par défaut à 3000)
# t : date du dernier match
# w : liste des h2h contre les autres joueurs
i=0

for player in data['Player1'].unique():
    if player not in index_joueurs_stats.keys():
        index_joueurs_stats[player] = [i,0,0,3000,0]
        i+=1
for player in data['Player2'].unique():
    if player not in index_joueurs_stats.keys():
        index_joueurs_stats[player] = [i,0,0,3000,0]
        i+=1

print('Nombre exact de joueurs : ',i)

###**Classement Elo**

In [ ]:
classement = {x: 1500 for x in index_joueurs_stats}
nb_matchs = {x: 0 for x in index_joueurs_stats}
K = {x : 40 for x in index_joueurs_stats}

def classement_elo(data,classement,nb_matchs):

  # Création d'une colonne elo_ranking
  elo_p1 = np.zeros((data.shape[0], 1))
  elo_p2 = np.zeros((data.shape[0], 1))
  proba_win_best_ranked = np.zeros((data.shape[0], 1))

  # On parcourt tous les matchs, puis on calcule la proba de gain du joueur le
  # mieux classé contre le moins bien
  for index, row in data.iterrows():
    player1, player2 = row[6], row[11]
    elo1, elo2 = classement[player1], classement[player2]
    K1, K2 = K[player1], K[player2]

    elo_p1[index, 0] = classement[player1]
    elo_p2[index, 0] = classement[player2]


    D = elo1 - elo2
    if abs(D) > 400:
      D = (D/abs(D))*400

    # Proba de gain du joueur le mieux classé sur le pire
    # Proba de gain de P1 sur P2
    p = 1/(1 + 10**(-D/400))

    proba_win_best_ranked[index, 0] = round(p,2)

    if nb_matchs[player1]<=30:
      K[player1] = 40
    elif elo1 >= 2250 or K1==10:
      K[player1] = 10
    else:
      K[player1] = 30

    if nb_matchs[player2]<=30:
      K[player2] = 40
    elif elo2 >= 2250 or K[player2]==10:
      K[player2] = 10
    else:
      K[player2] = 30




    # Détermination de W, résultat réel
    W = row[13]

    # Actualisation du classement
    classement[player1] = round(elo1 + K1*(W-p))
    classement[player2] = round(elo2 + K2*(p-W))

    nb_matchs[player1] += 1
    nb_matchs[player2] += 1



  return (elo_p1, elo_p2, proba_win_best_ranked)

elo_p1, elo_p2, proba_win_best_ranked = classement_elo(data,classement,nb_matchs)

# Insertion des colonnes elo_p1 et elo_p2
data['elo_P1'] = elo_p1
data['elo_P2'] = elo_p2
data['proba_elo'] = proba_win_best_ranked

###**Feature meilleur classement en carrière**

In [ ]:
data['Best_rank_p1'] = 0
data['Best_rank_p2'] = 0

for c, row in data.iterrows():
  p1 = row[6]
  p2 = row[11]
  if row[5]<row[27]:
    data.at[c, 'Best_rank_p1'] = row[5]
    index_joueurs_stats[p1][3] = row[5]
  else:
    data.at[c, 'Best_rank_p1'] = index_joueurs_stats[p1][3]

  if row[10]<index_joueurs_stats[p2][3]:
    data.at[c, 'Best_rank_p2'] = row[10]
    index_joueurs_stats[p2][3] = row[10]
  else:
    data.at[c, 'Best_rank_p2'] = index_joueurs_stats[p2][3]

###**Features du pourcentage de victoires (saison en cours)**

In [ ]:
index_joueurs_stats_clean = copy.deepcopy(index_joueurs_stats)
tableau_ratio = np.zeros(((len(data)),2))
date_limite = pd.to_datetime('2007-01-01')

for c,row in data.iterrows():
  if row[3]>=date_limite:
    date_limite = pd.to_datetime(date_limite) + pd.DateOffset(years=1)
    index_joueurs_stats = copy.deepcopy(index_joueurs_stats_clean)
  score_p1 = index_joueurs_stats[row[6]]
  score_p2 = index_joueurs_stats[row[11]]
  if score_p1[2]<5:
    tableau_ratio[c,0]=0.2
  else:
    tableau_ratio[c,0] =round(score_p1[1]/score_p1[2],2)
  if score_p2[2]<5:
    tableau_ratio[c,1]=0.2
  else:
    tableau_ratio[c,1] = round(score_p2[1]/score_p2[2],2)
  if row[13]==1:
    index_joueurs_stats[row[6]][0],index_joueurs_stats[row[6]][1],index_joueurs_stats[row[6]][2] = score_p1[0],score_p1[1]+1,score_p1[2]+1
    index_joueurs_stats[row[11]][0],index_joueurs_stats[row[11]][1],index_joueurs_stats[row[11]][2] = score_p2[0],score_p2[1],score_p2[2]+1
  else:
    index_joueurs_stats[row[6]][0],index_joueurs_stats[row[6]][1],index_joueurs_stats[row[6]][2] = score_p1[0],score_p1[1],score_p1[2]+1
    index_joueurs_stats[row[11]][0],index_joueurs_stats[row[11]][1],index_joueurs_stats[row[11]][2] = score_p2[0],score_p2[1]+1,score_p2[2]+1

data['ratio_p1_saison'] = tableau_ratio[:,0]
data['ratio_p2_saison'] = tableau_ratio[:,1]

## **Feature du pourcentage de vitoire (en carrière)**

In [ ]:
index_joueurs_stats = copy.deepcopy(index_joueurs_stats_clean)
tableau_ratio_1 = np.zeros(((len(data)),2))

for c,row in data.iterrows():
  score_p1 = index_joueurs_stats[row[6]]
  score_p2 = index_joueurs_stats[row[11]]
  if score_p1[2]<5:
    tableau_ratio_1[c,0]=0.2
  else:
    tableau_ratio_1[c,0] =round(score_p1[1]/score_p1[2],2)
  if score_p2[2]<5:
    tableau_ratio_1[c,1]=0.2
  else:
    tableau_ratio_1[c,1] = round(score_p2[1]/score_p2[2],2)
  if row[13]==1 :
    index_joueurs_stats[row[6]][0],index_joueurs_stats[row[6]][1],index_joueurs_stats[row[6]][2] = score_p1[0],score_p1[1]+1,score_p1[2]+1
    index_joueurs_stats[row[11]][0],index_joueurs_stats[row[11]][1],index_joueurs_stats[row[11]][2] = score_p2[0],score_p2[1],score_p2[2]+1
  else:
    index_joueurs_stats[row[6]][0],index_joueurs_stats[row[6]][1],index_joueurs_stats[row[6]][2] = score_p1[0],score_p1[1],score_p1[2]+1
    index_joueurs_stats[row[11]][0],index_joueurs_stats[row[11]][1],index_joueurs_stats[row[11]][2] = score_p2[0],score_p2[1]+1,score_p2[2]+1

data['ratio_p1_carrière'] = tableau_ratio_1[:,0]
data['ratio_p2_carrière'] = tableau_ratio_1[:,1]

**Fonction qui calcule la proba de gain avec le classement elo**

In [ ]:
def calcul_proba_elo(pts_elo1, pts_elo2, seuil):
  D = pts_elo1 - pts_elo2
  if abs(D) > seuil:
    D = (D/abs(D))*seuil

  # Proba de gain de P1 sur P2
  p = 1/(1 + 10**(-D/seuil))

  return (p, 1-p)

In [ ]:
!pip install fuzzywuzzy

**Fonction basé sur la distance de levenstein pour harmoniser les noms des joueurs entre le classement et le dataset**

In [ ]:
from fuzzywuzzy import fuzz

def abréviation_nom(nom_complet):
    parties_nom = nom_complet.split()
    nom_abrégé = parties_nom[-1]  + ' ' +''.join(parties_nom[0][0])+ '.'
    return nom_abrégé

def trouver_correspondance_nom(nom_inexact, liste_noms):
    correspondance_score_max = 0
    nom_correspondant = None
    for nom_exact in liste_noms:
        score = fuzz.partial_ratio(nom_inexact, nom_exact)
        if score > correspondance_score_max:
            correspondance_score_max = score
            nom_correspondant = nom_exact
    return nom_correspondant

In [ ]:
def distance_levenshtein(mot1, mot2):
    # On initialise une matrice de taille (longueur_mot1 + 1) x (longueur_mot2 + 1)
    dist = [[0] * (len(mot2) + 1) for _ in range(len(mot1) + 1)]

    # On initialise la première colonne avec les distances de 0 à la longueur du mot1
    for i in range(len(mot1) + 1):
        dist[i][0] = i

    # On initialise la première ligne avec les distances de 0 à la longueur du mot2
    for j in range(len(mot2) + 1):
        dist[0][j] = j

    # On calcule la distance de Levenshtein
    for i in range(1, len(mot1) + 1):
        for j in range(1, len(mot2) + 1):
            cost = 0 if mot1[i - 1] == mot2[j - 1] else 1
            dist[i][j] = min(dist[i - 1][j] + 1, dist[i][j - 1] + 1, dist[i - 1][j - 1] + cost)

    # La distance de Levenshtein est le dernier élément de la matrice
    return dist[len(mot1)][len(mot2)]

def trouver_correspondance_nom_2(nom_inexact, liste_noms):
    correspondance_score_min = float('inf')
    nom_correspondant = None
    for nom_exact in liste_noms:
        distance = distance_levenshtein(nom_inexact, nom_exact)
        if distance < correspondance_score_min:
            correspondance_score_min = distance
            nom_correspondant = nom_exact
    return nom_correspondant

# **Création du dictionnaire dont les clés sont les dates (chaque premier du mois entre le 01-01-2015 et le 01-10-2023) et les valeurs les classements aux dates correspondantes stockés sous forme de liste**

In [ ]:
import numpy as np

###cette fonction permet de creer le classement pour une date donnée

def classement_mois(date):
  # Ici on récupère les données relatives à chacun des joueurs à la date directement disponible dans le dataset
  # ie : pour un joueur : [classement_elo, ratio_saison, ratio_carrière]
  stats_joueurs_pour_proba = {}
  date_limite = pd.to_datetime(date)
  date_limite_formatee = date_limite.strftime('%d-%m-%Y')
  data_r = data[(data['Date']>=date_limite-timedelta(days=730)) & (data['Date']<=date_limite_formatee)]
  data_r = data_r.reset_index(drop=True)
  data_r['Date'] = data_r['Date'].dt.strftime('%d-%m-%Y')
  for index, ligne in data_r.iterrows():
    # Données joueur 1
    stats_joueurs_pour_proba[ligne[6]] = [ligne[24], ligne[29], ligne[31]]

    # Données joueur 2
    stats_joueurs_pour_proba[ligne[11]] = [ligne[25], ligne[30], ligne[32]]


  # On importe le classement ATP du 30/10/23 et on récupère les données de chaque joueur (100 premiers à l'ATP)
  # NB : Nécessairement il ne reste donc que les joueurs (ayant joué un match ou plus les deux dernières années) ET (présent dans le top 100 ATP à la date)
  atp_rkg = pd.read_csv(f'/content/drive/MyDrive/TIPE/classements atp/{date}.csv')
  atp_rkg = atp_rkg[:100]
  stats_joueurs_pour_proba_final = {}
  liste_joueurs = stats_joueurs_pour_proba.keys()
  print(liste_joueurs)
  cpt_index = 0
  for index, row in atp_rkg.iterrows():

    name = row[1]
    l = name.split(' ')
    if len(l) >= 2:
      name = trouver_correspondance_nom(abréviation_nom(name),liste_joueurs)
    if name in liste_joueurs:
      f_stats = stats_joueurs_pour_proba[name]
      # STATS : [index, points ATP, classement ATP actuel, points elo, meilleur classement carrière ATP, Ratio saison, Ratio carrière]
      stats = [cpt_index, row[4], row[0], f_stats[0], row[5]] + f_stats[1:]
      stats_joueurs_pour_proba_final[name] = stats
      cpt_index += 1
      #print(cpt_index,name,stats,stats_joueurs_pour_proba_final[name])
  print(stats_joueurs_pour_proba_final)




  ###Cette fonction ajoute au dictionnaire stats_joueurs_pour_proba_final le nombre de matchs gagnés sur chaque autre joueur sous forme d'une liste

  def get_h2h():
    n = len(stats_joueurs_pour_proba_final)

    for player1 in stats_joueurs_pour_proba_final:
        p1_h2h = [0 for _ in range(n)]
        cpt_p2 = 0

        for player2 in stats_joueurs_pour_proba_final:
          data_pour_h2h = data[((data['Player1'] == player1) & (data['Player2'] == player2 )) | ((data['Player1'] == player2) & (data['Player2'] == player1))]

          match_p1 = 0
          for index, row in data_pour_h2h.iterrows():
            # S'il est player1, mieux classé et winner = 1 OU player2, moins bien classé et winner = 0
            if (row[6] == player1 and row[13]==1) or (row[11]==player1 and row[13]==0):
              match_p1 += 1

          p1_h2h[cpt_p2] = match_p1
          cpt_p2 += 1
        stats_joueurs_pour_proba_final[player1].append(p1_h2h)

  get_h2h()


  ### Cette fonction calucle la proba de victoire d'un joueur j sur un joueur i en se basant sur la formule du classement élo adaptée à plusieurs statistiques

  def calcul_proba(player1, player2):
    stats1, stats2 = stats_joueurs_pour_proba_final[player1], stats_joueurs_pour_proba_final[player2]

    # Calcul proba elo
    p1_elo, p2_elo = calcul_proba_elo(stats1[3], stats2[3], 400)

    # Calcul proba ATP
    p1_atp, p2_atp = calcul_proba_elo(stats1[1], stats2[1], 3000)

    # Calcul proba classement ATP
    p1_catp, p2_catp = calcul_proba_elo(stats1[2], stats2[2], 40)

    # Calcul proba ratio saison
    p1_rs, p2_rs = calcul_proba_elo(stats1[5], stats2[5], 0.4)

    # Calcul proba h2h
    h2h_tot = stats1[7][stats2[0]] + stats2[7][stats1[0]]
    if h2h_tot >= 5:
      p1_h2h, p2_h2h = calcul_proba_elo(stats1[7][stats2[0]], stats2[7][stats1[0]], 10)

      # Calcul proba finale
      p = (50*p1_elo + 25*p1_rs + 25*p1_h2h)/100
    else:
      p = (70*p1_elo + 30*p1_rs)/100

    return (p, 1-p)

  ### Création de la matrice de transition
  dict_index_nom = {stats_joueurs_pour_proba_final[player][0]:player for player in stats_joueurs_pour_proba_final}
  print(dict_index_nom)

  n = len(stats_joueurs_pour_proba_final)
  matrice_transition = [[0 for _ in range(n)] for _ in range(n)]
  for j in range(n):
    somme = 0

    # On remplit la colonne j
    for i in range(n):
      if i !=j :
        pji = calcul_proba(dict_index_nom[j], dict_index_nom[i])[0]
        matrice_transition[i][j] = pji
        somme += pji
      else :
        matrice_transition[i][j] = 0
      if (i==0 or i==1 or i ==2) and (j==0 or j==1 or j==2) : print(matrice_transition[i][j],i,j)

    #On normalise la colonne
    for i in range(n):
      matrice_transition[i][j] /= somme

    matrice_transition = np.array(matrice_transition)


  n = 100 # dimension de la matrice de transition


  # Initialisation du vecteur
  Y = np.array([[1/n] for _ in range(n)])

  # Itération jusqu'à convergence
  for i in range(1000):
      tmp = np.dot(matrice_transition, Y)
      Y = tmp / np.linalg.norm(tmp)
      if i==0 or i == 9 or i==99 or i==999:
        print ([Y[0],Y[1],Y[2]])

  # Calcul de la valeur propre dominante
  valeur_propre_dominante = np.dot(tmp.T, Y) / np.dot(Y.T, Y)
  Y_normalized = Y / np.sum(Y)

  # Triez les indices des états en fonction des valeurs du vecteur propre
  classement_indices = np.argsort(Y_normalized, axis=0)[::-1]

  # Création du classement
  classement = [(indice, Y_normalized[indice][0]) for indice in classement_indices]

  C = []
  for i in range(len(classement)):
    C.append(dict_index_nom[classement[i][0][0]])

  return(C)



### Cette fonction permet finalement creer le dictionnaire souhaité
def obtenir_classements_personnels():
    classements_personnels = {}
    date_debut = datetime(2015, 1, 1)
    date_fin = datetime(2023, 10, 1)
    mois_actuel = date_debut

    while mois_actuel <= date_fin:
        # Vérifier si le mois actuel est dans la période à sauter
        if mois_actuel.year == 2020 and mois_actuel.month >= 4 and mois_actuel.month <= 8:
            # Passer au mois suivant sans rien faire
            mois_suivant = mois_actuel.replace(month=mois_actuel.month + 1)
            mois_actuel = mois_suivant
            continue

        # Calculer le premier jour du mois suivant
        mois_suivant = mois_actuel.replace(day=1)
        if mois_suivant.month == 12:
            mois_suivant = mois_suivant.replace(year=mois_suivant.year + 1, month=1)
        else:
            mois_suivant = mois_suivant.replace(month=mois_suivant.month + 1)

        # Obtenir le classement des joueurs pour le premier jour de ce mois
        date_str = mois_actuel.strftime('%d-%m-%Y')
        classement = classement_mois(date_str)

        # Ajouter ce classement à la liste des classements mensuels
        classements_personnels[mois_actuel.strftime('%Y-%m')] = classement

        # Passer au mois suivant
        mois_actuel = mois_suivant

    return classements_personnels

# Obtention des classements mensuels
classements_personnels = obtenir_classements_personnels()




In [ ]:
### je ne garde dans le dataset initial que les matchs à partir du 01-01-2015

data = data[data['Date']>='2015-01-01']
data = data.reset_index(drop=True)

#Supression des matchs impliquant des joueurs au delà de la 100ème place ATP(ceux qui n'ont pas de classement)
for c, row in data.iterrows():
  if (row[5]>100 or row[10]>100):
    data = data.drop([c], axis = 0)
data = data.reset_index(drop=True)

# **Création d'un nouveau dataset dans lequel les colonnes classement sont remplies avec mon classement**

In [ ]:
import pandas as pd
from datetime import datetime

def mettre_a_jour_classements(data, classements_personnels):
    # Créer une copie du dataset original pour le nouveau dataset
    data_my_rank = data.copy()

    # Convertir la colonne de dates en format datetime
    data_my_rank.iloc[:, 3] = pd.to_datetime(data_my_rank.iloc[:, 3])

    # Fonction pour obtenir le classement d'un joueur à une date donnée
    def obtenir_classement_joueur(joueur, date_match):
        date_str = date_match.strftime('%Y-%m')
        classement_mois = classements_personnels.get(date_str, [])
        try:
            classement_joueur = len(classement_mois) - classement_mois.index(joueur)
        except ValueError:
            classement_joueur = float('NaN')
        return classement_joueur

    # Mettre à jour les colonnes de classement des joueurs dans le nouveau dataset
    for index, row in data_my_rank.iterrows():
        date_match = row[3]
        nom_joueur1 = row[6]
        nom_joueur2 = row[11]
        classement_joueur1 = obtenir_classement_joueur(nom_joueur1, date_match)
        classement_joueur2 = obtenir_classement_joueur(nom_joueur2, date_match)
        data_my_rank.at[index, 'Player1Rank'] = classement_joueur1
        data_my_rank.at[index, 'Player2Rank'] = classement_joueur2

    return data_my_rank

data_my_rank = mettre_a_jour_classements(data, classements_personnels)


In [ ]:
data = data.dropna()
data = data.reset_index(drop=True)

data_my_rank = data_my_rank.dropna()
data_my_rank = data_my_rank.reset_index(drop=True)

# **Petite étude statistique**

In [ ]:
### Je crée un dictionnaire identique à celui créé précédemment mais pour le classement ATP

import os
classements_personnels = classements_personnels
def charger_classements_atp(classements_personnels):
    classements_atp = {}
    chemin_dossier = '/content/drive/MyDrive/TIPE/classements atp'
    for fichier in os.listdir(chemin_dossier):
        if fichier.endswith('.csv') and len(fichier) == 14:  # Vérifier si le fichier est au format jj-mm-aaaa.csv
            nom_fichier = os.path.join(chemin_dossier, fichier)
            df = pd.read_csv(nom_fichier)
            mois_annee = fichier[6:10] + '-' + fichier[3:5]  # Format 'aaaa-mm'
            # Sélectionner les 100 premiers joueurs du fichier et inverser l'ordre
            joueurs = df.iloc[:100, 1].tolist()[::-1]  # Sélectionne les 100 premiers, colonne 2 (index 1), et inverse
            # Aligner les noms de joueurs avec les classements personnels
            joueurs = [trouver_correspondance_nom(abréviation_nom(joueur), classements_personnels[mois_annee]) for joueur in joueurs]
            classements_atp[mois_annee] = joueurs
    return classements_atp
classements_atp = charger_classements_atp(classements_personnels)

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

def calculer_moyenne_variation(classements_personnels, classements_atp):
    variations_totales = []
    ecarts_types_totaux = []

    # Parcourir tous les mois disponibles
    for mois in classements_personnels.keys():
        if mois in classements_atp:
            classement_personnel_mois = classements_personnels[mois]
            classement_atp_mois = classements_atp[mois]
            # Calculer la variation de place pour chaque joueur
            variations_par_joueur = [abs(classement_personnel_mois.index(joueur) - classement_atp_mois.index(joueur)) for joueur in classement_personnel_mois]
            #Calculer l'écart type de la distribution de variation pour ce mois
            ecart_type_mois = np.std(variations_par_joueur)
            ecarts_types_totaux.append(ecart_type_mois)
            # Calculer la variation moyenne pour ce mois
            variation_moyenne = sum(variations_par_joueur) / len(variations_par_joueur)
            variations_totales.append(variation_moyenne)
    moyenne_variation = sum(variations_totales) / len(variations_totales)
    ecart_type_variation = sum(ecarts_types_totaux) / len(ecarts_types_totaux)
    return (moyenne_variation,ecart_type_variation,variations_totales,ecarts_types_totaux)

moyenne_variation,ecart_type_variation,variations_totales,ecarts_types_totaux = calculer_moyenne_variation(classements_personnels, classements_atp)

print(f"La moyenne[resp. l'écart type] des variations de classement est de : {moyenne_variation} [resp. {ecart_type_variation}]")

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

# Calculer les positions des mois à afficher
mois_a_afficher = [mois for i, mois in enumerate(mois) if i % 6 == 0]
mois_numeriques = range(len(mois))

# Tracer le graphique avec des paramètres ajustés
plt.figure(figsize=(14, 8))  # Ajustement de la taille du graphique

# Interpoler les données pour lisser la courbe
interpolateur = interp1d(mois_numeriques, variations_totales, kind='cubic')
mois_interpolés = np.linspace(mois_numeriques[0], mois_numeriques[-1], 1000)
variations_interpolées = interpolateur(mois_interpolés)

# Tracer la courbe lissée
plt.plot(mois_interpolés, variations_interpolées, '-', linewidth=2, color='#4169E1', label='Variation moyenne de classement')

# Paramètres d'axe et de légende
plt.xlabel('Mois')
plt.ylabel("Variation moyenne de classement")
plt.title('Variations moyennes de classement au fil du temps')
plt.xticks(mois_numeriques[::6], mois_a_afficher, rotation=45)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()
